In [4]:

def analyze_video(filename,modelname,iou,conf,imsz):
    import sys
    import ultralytics
    import supervision as sv
    from ultralytics import YOLO
    import numpy as np
    import cv2
    import cvzone
    import vehicle_count
    import json
    # from inference import get_model

    ultralytics.checks()
    print("supervision.__version__:", sv.__version__)

    # print("\n=======================================================================================")
    # print(f"Intiating traffic analysis for filename: {filename}")
    HOME = "/Users/matthew/Jupyter/Thesis"
    FILE_NAME = filename
    MODEL_NAME = modelname #"yolov9e"
    IOU=iou #0.8
    CONF=conf #0.3
    SOURCE_VIDEO_PATH = f"{HOME}/Dataset/{FILE_NAME}.MOV"
    TARGET_VIDEO_PATH = f"{HOME}/Outputs/YOLO/Realtime/{FILE_NAME}-{MODEL_NAME}-{imsz}-iou{IOU}-con{CONF}.mp4"
    TARGET_COUNT_PATH = f"{HOME}/Outputs/YOLO/Realtime/{FILE_NAME}-{MODEL_NAME}-{imsz}-iou{IOU}-con{CONF}.txt"
    vehicle_count.FILE_NAME = TARGET_COUNT_PATH
    MODEL = f"/Users/matthew/Jupyter/Thesis/Models/{MODEL_NAME}.pt"
    model = YOLO(MODEL)
    model.fuse()
    CLASS_NAMES_DICT = model.model.names
    selected_classes = [2, 3, 5, 7]
    # SETTINGS
    JSON_PATH = f"{HOME}/Dataset/Ground-Truth/{FILE_NAME}.json"
    with open(JSON_PATH) as json_file:
        json = json.load(json_file)
        lanes = json["lanes"]
        linezone = json["linezone"]
    vehicle_count.reset(TARGET_COUNT_PATH,json["vehicle_counts"])

    print("\n=======================================================================================")
    print(f"Intiating traffic analysis for {FILE_NAME}-{MODEL_NAME}-{imsz}-iou{IOU}-con{CONF}")

    # Inference Slicer
    # model_inf = get_model(model_id=MODEL_NAME)
    # image = cv2.imread(<SOURCE_IMAGE_PATH>)
    # def callback_inf_slicer(image_slice: np.ndarray) -> sv.Detections:
    #     results = model.infer(image_slice)[0]
    #     return sv.Detections.from_inference(results)

    # LINE_START_NORTH = sv.Point(50, 1200)
    # LINE_END_NORTH = sv.Point(1900, 1200)
    # LINE_START_SOUTH = sv.Point(1900, 1200)
    # LINE_END_SOUTH = sv.Point(3840-50, 1200)
    LINE_START_NORTH = sv.Point(10, 400)
    LINE_END_NORTH = sv.Point(633, 400)
    LINE_START_SOUTH = sv.Point(634, 400)
    LINE_END_SOUTH = sv.Point(1280-10, 400)
    sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)



    # VIDEO PROCESSING
    # create BYTETracker instance
    sv.ByteTrack()
    byte_tracker = sv.ByteTrack(track_activation_threshold=0.25, lost_track_buffer=30, minimum_matching_threshold=0.8, frame_rate=24)
    # byte_tracker = sv.ByteTrack()
    byte_tracker.reset()
    # create VideoInfo instance
    video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
    # create frame generator
    generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
    # create LineZone instance, it is previously called LineCounter class
    line_zone_north = sv.LineZone(start=LINE_START_NORTH, end=LINE_END_NORTH, triggering_anchors=[sv.Position.BOTTOM_LEFT,sv.Position.BOTTOM_RIGHT,sv.Position.BOTTOM_CENTER])
    line_zone_south = sv.LineZone(start=LINE_START_SOUTH, end=LINE_END_SOUTH, triggering_anchors=[sv.Position.BOTTOM_LEFT,sv.Position.BOTTOM_RIGHT,sv.Position.BOTTOM_CENTER])
    # create instance of BoxAnnotator
    # box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
    box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator(text_scale=0.5, text_padding=1, text_position=sv.Position.TOP_CENTER)
    # create instance of TraceAnnotator
    trace_annotator = sv.TraceAnnotator()
    # create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
    line_zone_annotator_north = sv.LineZoneAnnotator(text_scale=0.5, text_thickness=1, text_padding=2, display_out_count=False, custom_in_text="North")
    line_zone_annotator_south = sv.LineZoneAnnotator(text_scale=0.5, text_thickness=1, text_padding=2, display_in_count=False, custom_out_text="South")
    totalCountUp = []
    totalCountDown= []
    # define call back function to be used in video processing
    def callback(frame: np.ndarray, index:int) -> np.ndarray:

        # if index > 10:
        #     return False
        # frame = sv.resize_image(
        #     image=frame, resolution_wh=(1920, 1080), keep_aspect_ratio=True
        # )
        # model prediction on single frame and conversion to supervision Detections
        results = model(frame, verbose=False, device='mps', iou=IOU, conf=CONF, imgsz=imsz)[0]
        # results = model(frame, verbose=False, device='mps', classes=selected_classes)[0]
        detections = sv.Detections.from_ultralytics(results)
        # only consider class id from selected_classes define above
        detections = detections[np.isin(detections.class_id, selected_classes)]
        # tracking detections
        detections = byte_tracker.update_with_detections(detections)
        labels = [
            f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
            for confidence, class_id, tracker_id
            in zip(detections.confidence, detections.class_id, detections.tracker_id)
        ]
        annotated_frame = trace_annotator.annotate(
            scene=frame.copy(),
            detections=detections
        )
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame, detections=detections
        )
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame, detections=detections, labels=labels
        )
        # update line counter
        crossed_north, _ = line_zone_north.trigger(detections)
        _, crossed_south = line_zone_south.trigger(detections)

        if np.any(crossed_north):
            detections_crossed_north = detections[crossed_north]
            for detection in (detections_crossed_north):
                xyxy, _, conf, cls_id, tracker_id, cls_name = detection
                cls_name = cls_name['class_name']
                x1, y1, x2, y2 = xyxy
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w, h = x2 - x1, y2 - y1
                cx, cy = x1 + w // 2, y1 + h // 2
                if lanes["n1_start"][0] < cx < lanes["n1_end"][0] :
                    lane_id = "N1"
                elif lanes["n2_start"][0] < cx < lanes["n2_end"][0] :
                    lane_id = "N2"
                elif lanes["n3_start"][0] < cx < lanes["n3_end"][0] :
                    lane_id = "N3"
                else:
                    lane_id = "NX"
                # print(f'id:{tracker_id} cls:{cls_name} lane: {lane_id}')
                # crop = sv.crop_image(frame, [x1, y1, x2, y2])
                # cv2.imwrite("cropimg.jpg", crop)
                vehicle_count.update(annotated_frame, index, x1, y1, h, w, tracker_id, cls_name, conf, lane_id)

        if np.any(crossed_south):
            detections_crossed_south = detections[crossed_south]
            for detection in (detections_crossed_south):
                xyxy, _, conf, cls_id, tracker_id, cls_name = detection
                cls_name = cls_name['class_name']
                x1, y1, x2, y2 = xyxy
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w, h = x2 - x1, y2 - y1
                cx, cy = x1 + w // 2, y1 + h // 2
                if lanes["s1_start"][0] < cx < lanes["s1_end"][0] :
                    lane_id = "S1"
                elif lanes["s2_start"][0] < cx < lanes["s2_end"][0] :
                    lane_id = "S2"
                elif lanes["s3_start"][0] < cx < lanes["s3_end"][0] :
                    lane_id = "S3"
                else:
                    lane_id = "SX"
                vehicle_count.update(annotated_frame, index, x1, y1, h, w, tracker_id, cls_name, conf, lane_id)
        
        annotated_frame=line_zone_annotator_north.annotate(annotated_frame, line_counter=line_zone_north)
        annotated_frame=line_zone_annotator_south.annotate(annotated_frame, line_counter=line_zone_south)

        sv.draw_line(annotated_frame,start=sv.Point(lanes["n1_start"][0],lanes["n1_start"][1]),end=sv.Point(lanes["n1_end"][0],lanes["n1_end"][1]), color=sv.Color.BLUE)
        sv.draw_line(annotated_frame,start=sv.Point(lanes["n2_start"][0],lanes["n2_start"][1]),end=sv.Point(lanes["n2_end"][0],lanes["n2_end"][1]), color=sv.Color.RED)
        sv.draw_line(annotated_frame,start=sv.Point(lanes["n3_start"][0],lanes["n3_start"][1]),end=sv.Point(lanes["n3_end"][0],lanes["n3_end"][1]), color=sv.Color.BLUE)

        sv.draw_line(annotated_frame,start=sv.Point(lanes["s1_start"][0],lanes["s1_start"][1]),end=sv.Point(lanes["s1_end"][0],lanes["s1_end"][1]), color=sv.Color.BLUE)
        sv.draw_line(annotated_frame,start=sv.Point(lanes["s2_start"][0],lanes["s2_start"][1]),end=sv.Point(lanes["s2_end"][0],lanes["s2_end"][1]), color=sv.Color.RED)
        sv.draw_line(annotated_frame,start=sv.Point(lanes["s3_start"][0],lanes["s3_start"][1]),end=sv.Point(lanes["s3_end"][0],lanes["s3_end"][1]), color=sv.Color.BLUE)

        vehicle_count.progressbar(index,video_info.total_frames)

        # sv.plot_image(annotated_frame, (16,16))
        # cv2.imshow("Processing Video", annotated_frame)
        return annotated_frame

    # process the whole video
    sv.process_video(
        source_path = SOURCE_VIDEO_PATH,
        target_path = "",
        callback=callback
    )

    vehicle_count.end()

#  Run multiple files in loop and record output
if __name__ == "__main__":
    models = [
        # "yolov8n", # error
        # "yolov8s",
        # "yolov8m",
        # "yolov8l",

        # "yolov9t",
        # "yolov9s",
        # "yolov9m",
        # "yolov9c",
        
        # "yolov8x",
        # "yolov9e",
        # "yolov10x",
        # "yolo11x"
        
        "yolov10l",
    ]
    ious = [0.5]
    cons = [0.4]
    # [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    imszs = [
        # 480,
        # 704,
        # 640,
        960,
        # 1280
        ]
    video_files = [
        "DS1",
        "DS2",
        "DC1", 
        "DC2",
        "DR1",
        "DR2",
        "HR1",
        "HR2",
        # "EC1", 
        "FG1",
        "NC1",
        "NC2",
        # "TEST",
        # "DS3",
        # "DS4",
        # "DS5",
        ]
    for model in models:
        for video_file in video_files:
            for imsz in imszs:
                for iou in ious:
                    for conf in cons:
                        analyze_video(video_file, model,iou,conf,imsz)


Ultralytics 8.3.28 🚀 Python-3.12.3 torch-2.3.0 CPU (Apple M1 Max)
Setup complete ✅ (10 CPUs, 32.0 GB RAM, 368.0/460.4 GB disk)
supervision.__version__: 0.20.0
YOLOv10l summary (fused): 461 layers, 25,839,728 parameters, 0 gradients, 127.0 GFLOPs

Intiating traffic analysis for NC2-yolov10l-960-iou0.5-con0.4


[ERROR:0@20019.565] global cap.cpp:645 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:216: error: (-215:Assertion failed) !filename.empty() in function 'icvExtractPattern'




 (2879/2880)	N: 156	S: 81	99.97%	▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░